In [38]:
import os
import sys
from typing import Dict, Any, List
from pathlib import Path
import openai
from tqdm import tqdm
import random
import pandas as pd
from pydantic import BaseModel
sys.path.append('../')
from most import MostClient, Glossary, Catalog, GlossaryNGram, Item

In [48]:
client = MostClient(model_id="most-67ebccad691330291add5bdd")

In [5]:
catalog = Catalog(client)

In [29]:
ai = openai.AsyncClient(api_key="")

In [7]:
path = '/Users/kaspar_george/Downloads/Items.csv'

In [9]:
df = pd.read_csv(path)
df.head().T

,0,1,2,3,4
Наименование товара,Банкетка Аймо Cozy Milk,Банкетка Аймо Textile Blue,Банкетка Аймо Textile Grafit,Банкетка Аймо Textile Grey,Банкетка Аймо Textile Light


In [32]:
class DetailedItem(BaseModel):
    pronunciation: str
    description: str
    similar_colors: List[str]
    item_type: str
    item_category: str
    similar_items: List[str]

In [33]:
prompt = """
Ты - лучший продавец консультант в магазине.
Дано наименование товара. 
Твоя задача - составить произношение этого товара на русском языке (pronunciation)
Добавить мета-данные для этого товара, включая описание товара, похожие цвета, тип товара, и другое
"""

In [34]:
async def format_item(title):
    response = await ai.responses.parse(
        model="gpt-4o-mini",
        input=[
            {"role": "system", "content": prompt},
            {
                "role": "user",
                "content": title,
            },
        ],
        text_format=DetailedItem,
    )
    return response.output_parsed

In [31]:
await format_item(df.loc[0]['Наименование товара'])

DetailedItem(pronunciation='банкетка аймо кози милк', description='Банкетка Аймо Cozy Milk — элегантная и комфортабельная мебель, идеально подходящая для вашего интерьера. Она выполнена в нежных светлых тонах с текстурированной обивкой, что придает ей стильный вид. Подходит для гостиных, спален и прихожих.', similar_colors=['Слоновая кость', 'Светло-бежевый', 'Пудровый розовый', 'Светло-серый'], item_type='Мебель', similar_items=['Банкетка с обивкой', 'Диван', 'Кресло', 'Табурет'])

In [39]:
rows = list(df.iterrows())
random.shuffle(rows)

In [ ]:
for idx, row in tqdm(rows[23 + 19 + 138:]):
    title = row['Наименование товара']
    fitem = await format_item(title)
    catalog.add_items(Item(title=title,
                           pronunciation=fitem.pronunciation,
                           metadata={
                               "description": fitem.description,
                               "similar_colors": ", ".join(fitem.similar_colors),
                               "item_type": fitem.item_type,
                               "item_category": fitem.item_category,
                               "similar_items": ", ".join(fitem.similar_items),
                           }))


In [47]:
catalog.search_items("Зеленый стул")

[Item(title='Стул Лаури Olive', pronunciation='Стул Лаури Олив', metadata={'description': 'Стул Лаури Olive - это стильный и удобный стул, выполненный в современном дизайне. Его уникальный цвет оливы добавляет уют и изысканность в любой интерьер. Этот стул отлично подойдет как для домашнего использования, так и для кафе или ресторанов. Изготовлен из прочных материалов, обеспечивающих долговечность и комфорт.', 'similar_colors': 'Зеленый, Хаки, Оливковый, Мятный', 'item_type': 'Стул', 'item_category': 'Мебель', 'similar_items': 'Стул Эдвард, Стул Патрика, Стул Ницца, Стул Милано'}, id='6861a85acd3c8262cce13ec1'),
 Item(title='Стул Элиза Velvet Milk', pronunciation='Стул Элиза Вельвет Мilk', metadata={'description': 'Стул Элиза Velvet Milk — это стильный и элегантный предмет мебели, выполненный из мягкого велюрового материала цвета молока. Он прекрасно вписывается в любой интерьер, добавляя нотку изысканности и комфорта. Удобная форма спинки и сиденья обеспечивает максимальный комфорт во

In [6]:
catalog.list_items()

[]

In [5]:
glossary.list_ngrams()

[]

In [7]:
glossary.add_ngrams(GlossaryNGram(original=["Диван", "Ру"],
                                  pronunciation=["диван", "ру"],
                                  weight=10))
glossary.add_ngrams(GlossaryNGram(original=["Диван", "Ру"],
                                  pronunciation=["иван", "ру"],
                                  weight=5))

[GlossaryNGram(original=['Диван', 'Ру'], pronunciation=['иван', 'ру'], weight=5.0, id='6842be63cd3c8262cc94e57f')]

In [8]:
glossary.list_ngrams()

[GlossaryNGram(original=['Диван', 'Ру'], pronunciation=['диван', 'ру'], weight=10.0, id='6842bdbccd3c8262cc94d9e1'),
 GlossaryNGram(original=['Диван', 'Ру'], pronunciation=['иван', 'ру'], weight=5.0, id='6842be63cd3c8262cc94e57f')]

In [4]:
catalog.add_items(Item(title="Банкетка Аймо Cozy Milk",
                       pronunciation="банкетка аймо кози милк",
                       metadata={}))

[Item(title='Банкетка Аймо Cozy Milk', pronunciation='банкетка аймо кози милк', metadata={}, id='6842bf12cd3c8262cc94f12c')]

In [5]:
catalog.list_items()

[Item(title='Банкетка Аймо Cozy Milk', pronunciation='банкетка аймо кози милк', metadata={}, id='6842bf12cd3c8262cc94f12c')]